In [1]:
import random
import string
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics

In [25]:
import nltk

In [59]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

C:\Users\Balasubramanian\Anaconda2\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\Balasubramanian\Anaconda2\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
df = pd.read_csv('nlp_annotation.csv')

In [4]:
df.head()

,Incident,Summary,label
0,ALCOHOL Alcohol/Drug Overdose,A student was transported to a local hospital ...,3
1,DISTURBANCE Disturbing The Peace,A staff member reported that a security guard ...,1
2,PROPERTY Damaged Property,A non-USC male reported that the roof of his t...,1
3,THEFT-PETTY Theft Petty-Plain,A suspect removed the wheels from a bicycle.,2
4,TRAFFIC Traffic Collision Without Injuries,A non-USC male drove a cart into a light post ...,2


In [5]:
df = df.drop(['Incident'],axis=1)

In [6]:
df.head()

,Summary,label
0,A student was transported to a local hospital ...,3
1,A staff member reported that a security guard ...,1
2,A non-USC male reported that the roof of his t...,1
3,A suspect removed the wheels from a bicycle.,2
4,A non-USC male drove a cart into a light post ...,2


In [7]:
df_1K = df.iloc[:1000,:]

In [10]:
print df_1K['label'].value_counts()

2    651
3    153
1    140
4     45
5      8
Name: label, dtype: int64


In [9]:
df_1K = df_1K.dropna()
len(df_1K)

997

## Pre-processing
Integer of frequency counts

#### Count Dict

In [30]:
word_count_dict = {}

In [26]:
for index,row in df_1K.iterrows():
    s = row['Summary']
    #print nltk.word_tokenize(s)
    words = nltk.word_tokenize(s)
    words=[word.lower() for word in words if word.isalpha()]
    print(words)
    break

['A', 'student', 'was', 'transported', 'to', 'a', 'local', 'hospital', 'for', 'medical', 'treatment', 'following', 'an', 'alcohol', 'overdose', '.']


In [27]:
for index,row in df_1K.iterrows():
    s = row['Summary']
    #print nltk.word_tokenize(s)
    words = nltk.word_tokenize(s)
    words=[word.lower() for word in words if word.isalpha()]
    print(words)
    break

['a', 'student', 'was', 'transported', 'to', 'a', 'local', 'hospital', 'for', 'medical', 'treatment', 'following', 'an', 'alcohol', 'overdose']


In [31]:
for index,row in df_1K.iterrows():
    s = row['Summary']
    words = nltk.word_tokenize(s)
    words=[word.lower() for word in words if word.isalpha()]
    
    for i in words:
        try:
            word_count_dict[i]+=1
        except:
            word_count_dict[i]=1

In [32]:
word_count_dict

{'foul': 1,
 'four': 7,
 'protest': 1,
 'hanging': 1,
 'bike': 14,
 'under': 3,
 'teaching': 1,
 'activation': 2,
 'stabbed': 1,
 'school': 1,
 'motorcyclist': 1,
 'straws': 1,
 'unrelated': 1,
 'charter': 1,
 'bicycle': 131,
 'tires': 1,
 'second': 12,
 'street': 5,
 'sustaining': 18,
 'cesar': 1,
 'cooking': 2,
 'lights': 1,
 'above': 4,
 'conduct': 13,
 'sinks': 3,
 'reporter': 1,
 'met': 7,
 'protection': 1,
 'pursuit': 2,
 'hers': 1,
 'affiliated': 1,
 'obtained': 2,
 'balance': 1,
 'luggage': 1,
 'climbed': 1,
 'credit': 18,
 'smoke': 12,
 'permit': 10,
 'visible': 2,
 'guests': 3,
 'jackets': 1,
 'punching': 2,
 'unit': 4,
 'spoke': 1,
 'would': 3,
 'hospital': 129,
 'coins': 2,
 'occupying': 1,
 'chargers': 1,
 'overhead': 2,
 'panhandling': 1,
 'passport': 3,
 'fainting': 18,
 'aware': 2,
 'phone': 40,
 'hold': 1,
 'blocking': 1,
 'room': 5,
 'pursue': 1,
 'work': 3,
 'worn': 1,
 'locker': 1,
 'elbow': 2,
 'cited': 26,
 'currency': 10,
 'want': 1,
 'counseling': 2,
 'times': 6

In [37]:
wc_list = sorted(word_count_dict.items(), key=lambda x: x[1],reverse=True)

In [38]:
wc_list

[('a', 1831),
 ('the', 685),
 ('was', 615),
 ('to', 491),
 ('and', 390),
 ('student', 389),
 ('suspect', 372),
 ('for', 290),
 ('removed', 230),
 ('reported', 225),
 ('of', 217),
 ('location', 212),
 ('her', 209),
 ('after', 190),
 ('medical', 190),
 ('treatment', 190),
 ('an', 185),
 ('vehicle', 179),
 ('transported', 175),
 ('that', 174),
 ('at', 174),
 ('from', 159),
 ('member', 159),
 ('staff', 155),
 ('his', 153),
 ('bicycle', 131),
 ('hospital', 129),
 ('local', 128),
 ('on', 117),
 ('by', 110),
 ('male', 102),
 ('in', 96),
 ('he', 92),
 ('released', 79),
 ('female', 77),
 ('with', 69),
 ('were', 68),
 ('while', 68),
 ('she', 67),
 ('university', 65),
 ('arrested', 62),
 ('examined', 61),
 ('into', 59),
 ('unknown', 58),
 ('damage', 57),
 ('causing', 57),
 ('alarm', 54),
 ('activated', 48),
 ('missing', 48),
 ('complaining', 46),
 ('it', 46),
 ('two', 46),
 ('scene', 45),
 ('injuring', 43),
 ('fire', 43),
 ('suffering', 42),
 ('parked', 41),
 ('phone', 40),
 ('drove', 37),
 ('be'

In [42]:
wc_dict = {}
ct_no = 1
for item in wc_list:
    wc_dict[item[0]] = ct_no
    ct_no+=1

In [44]:
wc_dict['was']

3

In [47]:
sorted(wc_dict.items(), key=lambda x: x[1])

[('a', 1),
 ('the', 2),
 ('was', 3),
 ('to', 4),
 ('and', 5),
 ('student', 6),
 ('suspect', 7),
 ('for', 8),
 ('removed', 9),
 ('reported', 10),
 ('of', 11),
 ('location', 12),
 ('her', 13),
 ('after', 14),
 ('medical', 15),
 ('treatment', 16),
 ('an', 17),
 ('vehicle', 18),
 ('transported', 19),
 ('that', 20),
 ('at', 21),
 ('from', 22),
 ('member', 23),
 ('staff', 24),
 ('his', 25),
 ('bicycle', 26),
 ('hospital', 27),
 ('local', 28),
 ('on', 29),
 ('by', 30),
 ('male', 31),
 ('in', 32),
 ('he', 33),
 ('released', 34),
 ('female', 35),
 ('with', 36),
 ('were', 37),
 ('while', 38),
 ('she', 39),
 ('university', 40),
 ('arrested', 41),
 ('examined', 42),
 ('into', 43),
 ('unknown', 44),
 ('damage', 45),
 ('causing', 46),
 ('alarm', 47),
 ('activated', 48),
 ('missing', 49),
 ('complaining', 50),
 ('it', 51),
 ('two', 52),
 ('scene', 53),
 ('injuring', 54),
 ('fire', 55),
 ('suffering', 56),
 ('parked', 57),
 ('phone', 58),
 ('drove', 59),
 ('be', 60),
 ('unspecified', 61),
 ('side', 62

In [62]:
len(wc_dict)

1419

#### Changing the input to freq-count form

In [48]:
X_list = []
y_list = []
for index,row in df_1K.iterrows():
    y_list.append(int(row['label']))
    iwords = []
    s = row['Summary']
    words = nltk.word_tokenize(s)
    words=[word.lower() for word in words if word.isalpha()]
    for item in words:
        iwords.append(wc_dict[item])
    X_list.append(iwords)

In [50]:
X_list

[[1, 6, 3, 19, 4, 1, 28, 27, 8, 15, 16, 90, 17, 77, 103],
 [1,
  24,
  23,
  10,
  20,
  1,
  458,
  433,
  126,
  504,
  66,
  80,
  277,
  136,
  1,
  389,
  256,
  66,
  867,
  1053,
  22,
  2,
  177,
  68,
  2,
  12],
 [1,
  31,
  10,
  20,
  2,
  463,
  11,
  25,
  618,
  3,
  70,
  129,
  51,
  67,
  17,
  557,
  1377,
  1353,
  99,
  33,
  3,
  92,
  29,
  2,
  563,
  187,
  11,
  2,
  12],
 [1, 7, 9, 2, 478, 22, 1, 26],
 [1, 31, 59, 1, 471, 43, 1, 300, 385, 68, 2, 12, 46, 74, 45],
 [102,
  121,
  188,
  1,
  6,
  5,
  25,
  648,
  68,
  2,
  12,
  5,
  871,
  189,
  147,
  129,
  2,
  6,
  715,
  4,
  377,
  2,
  121,
  288,
  66,
  112,
  277,
  32,
  2,
  175,
  2,
  121,
  88,
  9,
  2,
  6,
  73,
  58,
  5,
  25,
  648,
  186,
  5,
  153,
  29,
  131],
 [1, 31, 3, 41, 8, 330, 400],
 [1, 7, 185, 4, 1024, 395, 52, 279, 4, 434, 297, 4, 2, 12],
 [1, 7, 3, 41, 8, 134, 1, 35, 18, 22, 2, 12],
 [1, 7, 9, 17, 1346],
 [1, 7, 282, 17, 576, 18, 5, 459, 2, 475, 107, 145, 63, 510, 223, 1

In [51]:
X = np.array(X_list)
y = np.array(y_list)

In [52]:
X

array([list([1, 6, 3, 19, 4, 1, 28, 27, 8, 15, 16, 90, 17, 77, 103]),
       list([1, 24, 23, 10, 20, 1, 458, 433, 126, 504, 66, 80, 277, 136, 1, 389, 256, 66, 867, 1053, 22, 2, 177, 68, 2, 12]),
       list([1, 31, 10, 20, 2, 463, 11, 25, 618, 3, 70, 129, 51, 67, 17, 557, 1377, 1353, 99, 33, 3, 92, 29, 2, 563, 187, 11, 2, 12]),
       list([1, 7, 9, 2, 478, 22, 1, 26]),
       list([1, 31, 59, 1, 471, 43, 1, 300, 385, 68, 2, 12, 46, 74, 45]),
       list([102, 121, 188, 1, 6, 5, 25, 648, 68, 2, 12, 5, 871, 189, 147, 129, 2, 6, 715, 4, 377, 2, 121, 288, 66, 112, 277, 32, 2, 175, 2, 121, 88, 9, 2, 6, 73, 58, 5, 25, 648, 186, 5, 153, 29, 131]),
       list([1, 31, 3, 41, 8, 330, 400]),
       list([1, 7, 185, 4, 1024, 395, 52, 279, 4, 434, 297, 4, 2, 12]),
       list([1, 7, 3, 41, 8, 134, 1, 35, 18, 22, 2, 12]),
       list([1, 7, 9, 17, 1346]),
       list([1, 7, 282, 17, 576, 18, 5, 459, 2, 475, 107, 145, 63, 510, 223, 147]),
       list([1, 7, 9, 1, 123, 72, 194]),
       list([1, 24

In [54]:
maxx = 0
maxs = ""
for index,row in df_1K.iterrows():
    s = row['Summary']
    l = len(s)
    if(l>maxx):
        maxx = l
        maxs = s
print 'Max len in document:',maxx
print maxs

Max len in document: 424
A student reported that while she was riding on a Metro train a suspect snatched her cellular phone from her hand and fled out the door as the train stopped at the next station. The suspect then punched the student in the face when she attempted to pursue the subject. The student fell to the ground and the suspect and her companions removed a second phone and a set of keys from the student's purse before fleeing on foot.


In [ ]:
from keras.utils.np_utils import to_categorical
cy = to_categorical(y, num_classes=None)
cy

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, cy, test_size=0.3)

In [74]:
type(X_train)

numpy.ndarray

In [68]:
len(X_train)

697

In [75]:
# truncate and pad input sequences
max_review_length = 424
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [76]:
X_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [87]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(1420, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(6, activation='sigmoid'))

In [88]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 424, 32)           45440     
_________________________________________________________________
lstm_5 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 606       
Total params: 99,246
Trainable params: 99,246
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
model.fit(X_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
697/697 [==============================] - 42s 60ms/step - loss: 1.7579 - acc: 0.5911
Epoch 2/3
697/697 [==============================] - 22s 31ms/step - loss: 1.3367 - acc: 0.6456
Epoch 3/3
697/697 [==============================] - 24s 34ms/step - loss: 1.0755 - acc: 0.6456


In [90]:
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 67.00%
